<a href="https://colab.research.google.com/github/jjackson1994/molecular_simulations/blob/main/Melika/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import os.path as osp
from sentence_transformers import SentenceTransformer
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import LinkNeighborLoader
from torch import Tensor
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import os
import torch

In [ ]:

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git





2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [ ]:
#!pip install sentence-transformers
#!pip install torch_geometric

In [ ]:
p_path= '/content/1ARG.csv'

df = pd.read_csv(p_path)

In [ ]:
df.head()

,Unnamed: 0,PdbID,Res1,Res2,ChainRes1,ChainRes2,DensityRes1,DensityRes2,AA1,AA2,NativeEnergy,DecoyEnergy,SDEnergy,FrstIndex,Welltype,FrstState
0,0,1ARG,5,7,A,A,3.304,0.057,M,E,-1.267,-1.571,0.444,-0.685,long,neutral
1,1,1ARG,5,7,B,B,3.555,0.148,M,E,-1.422,-1.571,0.444,-0.336,long,neutral
2,2,1ARG,5,8,A,A,3.304,0.000,M,N,-1.291,-1.571,0.444,-0.631,long,neutral
3,3,1ARG,5,8,B,B,3.555,0.000,M,N,-1.295,-1.571,0.444,-0.622,long,neutral
4,4,1ARG,5,9,A,A,3.304,3.957,M,I,-2.553,-1.571,0.444,2.210,long,minimally


In [ ]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [ ]:

class SequenceEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

here are nodes are the residues. their features are their `Chain name`, and their `amino acid` type. they have their residue number as their index. we defined two types of nodes here with Res1 and Res2:

In [ ]:
Res1_x, Res1_mapping = load_node_csv(
    p_path, index_col='Res1', encoders={
        'ChainRes1': SequenceEncoder(),
        'AA1': SequenceEncoder()
    })

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

In [ ]:
Res2_x, Res2_mapping = load_node_csv(
    p_path, index_col='Res2', encoders={
        'ChainRes2': SequenceEncoder(),
        'AA2': SequenceEncoder()
    })

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

With this, we are ready to initialize our HeteroData object and pass two node types into it:

In [ ]:
data = HeteroData()

data['R1'].x = Res1_x # both of the nodes have features
data['R2'].x = Res2_x

data["R1"].node_idx = torch.arange(len(df['Res1']))
data["R2"].node_idx = torch.arange(len(df['Res2']))

print(data)


HeteroData(
  R1={
    x=[5830, 768],
    node_idx=[5830],
  },
  R2={
    x=[5830, 768],
    node_idx=[5830],
  }
)


It's time for edge now. we need to connect the nodes together. our edge here is the frustration index:

* Here, `src_index_col` and `dst_index_col` define the index columns of source and destination nodes, respectively.
* We further make use of the node-level mappings `src_mapping` and `dst_mapping` to ensure that raw indices are mapped to the correct consecutive indices in our final representation.

In [ ]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

Similarly to `load_node_csv()`, encoders are used to return additional edge-level feature information. For example, for loading the frustration index from the column in csv file, we can define an IdentityEncoder that simply converts a `list of floating-point values` into a `PyTorch tensor`:

In [ ]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
edge_index, edge_label = load_edge_csv(
    p_path,
    src_index_col='Res1',
    src_mapping=Res1_mapping,
    dst_index_col='Res2',
    dst_mapping=Res2_mapping,
    encoders={'FrstIndex': IdentityEncoder(dtype=torch.long)},
)

data['R1', 'F_index', 'R2'].edge_index = edge_index
data['R1', 'F_index', 'R2'].edge_label = edge_label


print(data.metadata)

<bound method HeteroData.metadata of HeteroData(
  R1={
    x=[5830, 768],
    node_idx=[5830],
  },
  R2={
    x=[5830, 768],
    node_idx=[5830],
  },
  (R1, F_index, R2)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  }
)>


In [ ]:
data

HeteroData(
  R1={
    x=[5830, 768],
    node_idx=[5830],
  },
  R2={
    x=[5830, 768],
    node_idx=[5830],
  },
  (R1, F_index, R2)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  }
)

In [ ]:
data['R1', 'F_index', 'R2'].edge_label[:10]

tensor([[0],
        [0],
        [0],
        [0],
        [2],
        [2],
        [1],
        [0],
        [0],
        [1]])

This **HeteroData object** is the native format of **heterogeneous graphs** in **PyG** and can be used as input for **heterogeneous graph models**.

# Heterogeneous Graph Learning:

Heterogeneous graphs come with different types of information attached to nodes and edges. Thus, a single node or edge feature tensor cannot hold all node or edge features of the whole graph, due to differences in type and dimensionality. Instead, a set of types need to be specified for nodes and edges, respectively, each having its own data tensors. As a consequence of the different data structure, the **message passing formulation** changes accordingly, allowing the computation of message and update function conditioned on **node** or **edge type**.

#### Note: here we have two node types.

we have created a data object of type torch_geometric.data.HeteroData, for which we defined `node feature tensors`, `edge index tensors` and `edge feature tensors` individually for each type.

Node or edge tensors will be automatically created upon first access and indexed by string keys. Node types are identified by a single string while edge types are identified by using a triplet (source_node_type, edge_type, destination_node_type) of strings: the edge type identifier and the two node types between which the edge type can exist. As such, the data object allows different feature dimensionalities for each type.

Node or edge tensors will be automatically created upon first access and indexed by string keys. Node types are identified by a single string while edge types are identified by using a triplet `(source_node_type, edge_type, destination_node_type)` of strings. the edge type identifier and the two node types between which the edge type can exist.

##### The data object can be transferred between devices as usual:

In [ ]:
#data = data.to('cuda:0')
#data = data.cpu()

## Why not a Homogenious model:

Standard Message Passing GNNs (MP-GNNs) can not trivially be applied to heterogeneous graph data, as node and edge features from different types can not be processed by the same functions due to differences in feature type. A natural way to circumvent this is to implement message and update functions individually for each edge type. During runtime, the MP-GNN algorithm would need to iterate over edge type dictionaries during message computation and over node type dictionaries during node updates.

In [ ]:
data = T.ToUndirected()(data)

In [ ]:
data

HeteroData(
  R1={
    x=[5830, 768],
    node_idx=[5830],
  },
  R2={
    x=[5830, 768],
    node_idx=[5830],
  },
  (R1, F_index, R2)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  },
  (R2, rev_F_index, R1)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  }
)

In [ ]:
data.edge_index = data["R1", "F_index", "R2"].edge_index
data.y = data["R1", "F_index", "R2"].edge_label.squeeze()

In [ ]:
data

HeteroData(
  edge_index=[2, 5830],
  y=[5830],
  R1={
    x=[5830, 768],
    node_idx=[5830],
  },
  R2={
    x=[5830, 768],
    node_idx=[5830],
  },
  (R1, F_index, R2)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  },
  (R2, rev_F_index, R1)={
    edge_index=[2, 5830],
    edge_label=[5830, 1],
  }
)

In [ ]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("R1", "F_index", "R2"),
    rev_edge_types=("R2", "rev_F_index", "R1"),
)
train_data, val_data, test_data = transform(data)

In [ ]:
train_data.edge_index = train_data["R1", "F_index", "R2"].edge_index
val_data.edge_index = val_data["R1", "F_index", "R2"].edge_index
test_data.edge_index = test_data["R1", "F_index", "R2"].edge_index

In [ ]:
train_data.y = train_data["R1", "F_index", "R2"].edge_label.squeeze()
val_data.y = val_data["R1", "F_index", "R2"].edge_label.squeeze()
test_data.y = test_data["R1", "F_index", "R2"].edge_label.squeeze()

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv, to_hetero

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=128, out_channels=64)
model = to_hetero(model, data.metadata(), aggr='sum')

The model will embed the node features to a size of 64 vectors, and these vectors will be feeded into the following linkp model to predict the rating.

In [ ]:
import torch.nn.functional as F

class LinkP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(LinkP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):

        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
        x = self.lins[-1](x)
        return x
linkp=LinkP(64,64,1)

## Batch :

here we considered a limited number of neighbours around the one nodes(in this example we considered 15 nodes, which really make sense since the connections around one amino acid don't usually surpass this amount.)

In [ ]:
#trainloader
edge_label_index = train_data["R1", "F_index", "R2"].edge_label_index
edge_label = train_data["R1", "F_index", "R2"].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("R1", "F_index", "R2"), edge_label_index),
    edge_label= edge_label,
    batch_size=32,
    shuffle=True,
)



#valloader
edge_label_index = val_data["R1", "F_index", "R2"].edge_label_index
edge_label = val_data["R1", "F_index", "R2"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("R1", "F_index", "R2"), edge_label_index),
    edge_label= edge_label,
    batch_size=32,
    shuffle=True,
)


#Testloader
edge_label_index = test_data["R1", "F_index", "R2"].edge_label_index
edge_label = test_data["R1", "F_index", "R2"].edge_label
test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("R1", "F_index", "R2"), edge_label_index),
    edge_label= edge_label,
    batch_size=32,
    shuffle=True,
)

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:59: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from sklearn.metrics import roc_auc_score
import tqdm
l = []
# Training function
def train(model, linkp, optimizer, data_loader, device):
    model.train()
    linkp.train()
    total_loss = 0


    for data in tqdm.tqdm(data_loader):
        data.edge_index = data["R1", "F_index", "R2"].edge_index
        data = data.to(device)
        optimizer.zero_grad()

        # Forward pass through GNN model
        x = model(data.x_dict, data.edge_index_dict)


        # Get link prediction scores from LinkP model
        R1id = data["R1", "F_index", "R2"].edge_label_index[0]
        R2id = data["R1", "F_index", "R2"].edge_label_index[1]
        edge_scores = linkp(x['R1'][R1id,:], x['R2'][R2id,:])


        # Compute binary cross-entropy loss
        loss = F.binary_cross_entropy_with_logits(edge_scores, data["R1", "F_index", "R2"].edge_label.float())

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        l.append(total_loss)


    return total_loss / len(data_loader)


# Validation function
def evaluate(model, linkp, data_loader, device):
    model.eval()
    linkp.eval()
    preds = []
    ground_truths = []
    total_loss = 0

    for data in tqdm.tqdm(data_loader):
        data = data.to(device)
        with torch.no_grad():
            x = model(data.x_dict, data.edge_index_dict)
            R1id = data["R1", "F_index", "R2"].edge_label_index[0]
            R2id = data["R1", "F_index", "R2"].edge_label_index[1]
            edge_scores = linkp(x['R1'][R1id,:], x['R2'][R2id,:])

        loss = F.binary_cross_entropy_with_logits(edge_scores, data["R1", "F_index", "R2"].edge_label.float())
        optimizer.step()
        total_loss += loss.item()



    return total_loss / len(data_loader)


# Test function
def test(model, linkp, data_loader, device):
    model.eval()
    linkp.eval()
    preds = []
    ground_truths = []
    total_loss = 0

    for data in tqdm.tqdm(data_loader):
        data = data.to(device)
        with torch.no_grad():
            x = model(data.x_dict, data.edge_index_dict)
            R1id = data["R1", "F_index", "R2"].edge_label_index[0]
            R2id = data["R1", "F_index", "R2"].edge_label_index[1]
            edge_scores = linkp(x['R1'][R1id,:], x['R2'][R2id,:])

        loss = F.binary_cross_entropy_with_logits(edge_scores, data["R1", "F_index", "R2"].edge_label.float())
        optimizer.step()
        total_loss += loss.item()



    return total_loss / len(data_loader)



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gnn_model = GNN(hidden_channels=128, out_channels=64).to(device)
linkp = LinkP(64, 64, 1).to(device)
optimizer = torch.optim.Adam(
    list(gnn_model.parameters()) + list(linkp.parameters()), lr=0.01
)

train(model, linkp, optimizer, train_loader, device)


100%|██████████| 44/44 [00:00<00:00, 57.16it/s]


0.36234208281067287

In [ ]:
evaluate(model, linkp, val_loader, device)

100%|██████████| 55/55 [00:00<00:00, 96.96it/s]


0.5357667484066703

In [ ]:
test(model, linkp, test_loader, device)

100%|██████████| 55/55 [00:00<00:00, 90.91it/s]


0.8755594578656283

In [ ]:
# after using train_loader, val_loader, and test_loader DataLoader objects
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gnn_model = GNN(hidden_channels=128, out_channels=64).to(device)
linkp_model = LinkP(64, 64, 1).to(device)

optimizer = torch.optim.Adam(
    list(gnn_model.parameters()) + list(linkp_model.parameters()), lr=0.01
)

num_epochs = 10
best_val_auc = 0.0

for epoch in range(num_epochs):
    train_loss = train(model, linkp, optimizer, train_loader, device)
    val_auc = evaluate(model, linkp, val_loader, device)

    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_auc =test(model, linkp, test_loader, device)


    print(f'Epoch: {epoch+1:03d}, Train Loss: {train_loss:.4f}, Val loss: {val_auc:.4f}')

print(f'Test loss: {test_auc:.4f}')

100%|██████████| 55/55 [00:00<00:00, 92.76it/s]


Epoch: 001, Train Loss: 0.5705, Val loss: 1.1757


100%|██████████| 55/55 [00:00<00:00, 98.53it/s]


Epoch: 002, Train Loss: 0.5722, Val loss: 1.1730


100%|██████████| 55/55 [00:00<00:00, 96.47it/s]


...

Epoch: 008, Train Loss: 0.5755, Val loss: 1.1730


100%|██████████| 55/55 [00:00<00:00, 96.88it/s]


Epoch: 009, Train Loss: 0.5738, Val loss: 1.1704


100%|██████████| 55/55 [00:00<00:00, 96.85it/s]

Epoch: 010, Train Loss: 0.5747, Val loss: 1.1722
Test loss: 1.1645
